In [ ]:
# Import Libraries

import numpy as np
import pandas as pd
import random
import cv2
import os
import glob

import math 
import datetime
import time

import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50V2, VGG16
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Add, Activation, ZeroPadding2D, AveragePooling2D
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout, BatchNormalization

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV



%matplotlib inline

In [ ]:
# Dataset Directory
DIRECTORY = r'skin-lesions'

# Dataset Directory
#DIRECTORY = r'skin-lesions-HAM10000'

# Dataset Directory
#DIRECTORY = r'skin-lesions-ISIC2017'

# Categories of Dataset
CATEGORIES = ['melanoma', 'nevus', 'seborrheic_keratosis']

In [ ]:
IMAGE_SIZE = 224
lesions_data = []

for category in CATEGORIES:
    
    # Joins Categories to the Directory Folder
    new_folder = os.path.join(DIRECTORY, category)
    
    # Assign Lables to Data Categories (0: melanoma, 1: nevus, 2: seborrheic_keratosis)
    label = CATEGORIES.index(category)
    
    # Join Images with Relevant Category Folder
    for img in os.listdir(new_folder):
        img_path = os.path.join(new_folder, img)
        
        # Read Each Image as Array
        img_array = cv2.imread(img_path)
        
        # Resize the Each Image to 256 x 256
        img_array = cv2.resize(img_array, (IMAGE_SIZE, IMAGE_SIZE))
      
        # Combine all the Images and Labels in a Single Directory
        lesions_data.append([img_array, label])

In [ ]:
len(lesions_data)

In [ ]:
# Plot a Bar Chart to Show Total Images in Each Class

plot_list = []

for i in lesions_data:
    if(i[1] == 0):
        plot_list.append("Melanoma")
    elif(i[1] == 1):
        plot_list.append("Nevus")
    else:
        plot_list.append("Seborrheic Keratosis")
        
ax = sns.countplot(plot_list)
plt.title("Total Images in Each Class")
for p in ax.patches:
   ax.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01))

In [ ]:
# Pie Chart to Show the Porportion of Each Class

# Count occurrences of each category
counts = {}

for category in plot_list:
    counts[category] = counts.get(category, 0) + 1

# Prepare data for the pie chart
categories = counts.keys()
category_counts = counts.values()

# Create the pie chart
plt.figure(figsize=(6, 6))
plt.pie(category_counts, labels=categories, autopct='%1.1f%%')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.title('Skin Lesions Distribution')
plt.show()

In [ ]:
# Extract Features and Labels of Data

X = []
y = []

for features, labels in lesions_data:
    X.append(features)
    y.append(labels)

In [ ]:
len(X)

In [ ]:
len(y)

In [ ]:
# Convert Features into Array 

X = np.array(X)

In [ ]:
# Features Sahpe
X.shape

In [ ]:
# Convert Labels into Array

y = np.array(y)

In [ ]:
# Labels Shape
y.shape

In [ ]:
# List of all Primary Lables
np.unique(y)

In [ ]:
'''
# Define the number of folds
k = 5

# Create a StratifiedKFold object to perform k-fold cross-validation
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Initialize a list to store the performance metrics for each fold
fold_metrics = []

# Iterate over each fold
for train_index, val_index in kf.split(X, y):
    
    # Split the data into training and validation sets for the current fold
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Proposed SkinLesNet CNN Model

    model = Sequential()

    # 1st Convolutional Input Layer
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)))
    model.add(MaxPooling2D((2,2)))

    # 3rd Convolutional Layer
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))

    # 3rd Convolutional Layer
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))

    # Flatten Layer 
    model.add(Flatten())

    # Hidden Layer
    model.add(Dense(128, activation='relu'))

    # Output layer
    model.add(Dense(3, activation='softmax'))
    
    # Compiling the Sequential Model

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, ema_momentum=0.99)

    model.compile(loss='sparse_categorical_crossentropy',  optimizer=optimizer, metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

    # Evaluate the model on the validation set
    metrics = model.evaluate(X_val, y_val)

    # Store the metrics for the current fold
    fold_metrics.append(metrics)

# Calculate the average performance metrics across all folds
avg_metrics = np.mean(fold_metrics, axis=0)
print("Average Performance Metrics:")
print("Loss:", avg_metrics[0])
print("Accuracy:", avg_metrics[1])
'''

In [ ]:
# Split Data into Train and Test Sets (80:20)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Reshape Labels
y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

In [ ]:
print("Training Features: ", X_train.shape)
print("Testing Features: ", X_test.shape)

print("Training Labels: ", y_train.shape)
print("Testing Labels: ", y_test.shape)

In [ ]:
# Plot first 15 Images from Training Dataset

# Define Rows and Columns for Subplot
rows, columns = 5, 5

# Define Figure Size
fig=plt.figure(figsize=(12, 12))

# Visualize Random Images
for i in range(1, 15 +1):
    
    fig.add_subplot(rows, columns, i)
    
    plt.imshow(X_train[i-1])
    plt.xticks([])
    plt.yticks([])
    
    # Define Labels
    if y_train[i-1] == 0:
        label_name = "Melanoma"
    elif y_train[i-1] == 1:
        label_name = "Nevus"
    else:
        label_name = "Seborrheic Keratosis"
    plt.title("{}"
          .format(label_name))
    
plt.show()

In [ ]:
# Rescale Images to Normalize Pixel Values (0 - 255)

X_train = X_train.astype('float32')/255.0
X_test  = X_test.astype('float32')/255.0

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
# Plot Any Random Image from Training Data

idx = random.randint(0, len(X_train))
plt.imshow(X_train[idx, :])

In [ ]:
'''
# Proposed SkinLesNet CNN Model

model = Sequential()

# 1st Convolutional Input Layer
model.add(Conv2D(32, kernel_size=(3,3), padding='same', activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

# 2nd Convolutional Layer
model.add(Conv2D(32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# 3rd Convolutional Layer
model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# 4th Convolutional Layers
model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Flatten Layer 
model.add(Flatten())

# Hidden Layer
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))

# Output layer
model.add(Dense(3, activation='softmax'))
'''

# SkinLesNet Proposed Model

In [ ]:
# Proposed SkinLesNet CNN Model

start = datetime.datetime.now()

model = Sequential()

# 1st Convolutional Input Layer
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(MaxPooling2D((2,2)))

# 2nd Convolutional Input Layer
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

# 3rd Convolutional Layer
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

# 4th Convolutional Layer
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Dropout(0.5))

# Flatten Layer 
model.add(Flatten())

# Hidden Layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(3, activation='softmax'))

In [ ]:
# Display Model Summary

model.summary()

In [ ]:
plot_model(model, to_file='SkimLesNet.png')

In [ ]:
'''
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32

# Create an ImageDataGenerator instance for data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Create a data generator for the training set with data augmentation
train_datagen = datagen.flow(X_train, y_train, batch_size=batch_size, shuffle=True)

# Create a separate data generator for the test set without data augmentation
test_datagen = ImageDataGenerator().flow(X_test, y_test, batch_size=batch_size)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model using data augmentation
model.fit(train_datagen,
          epochs=10,
          steps_per_epoch=len(X_train) // batch_size,
          validation_data=test_datagen,
          validation_steps=len(X_test) // batch_size)
'''

In [ ]:
# Compiling the Sequential Model

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, ema_momentum=0.99)

model.compile(loss='sparse_categorical_crossentropy',  optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# Training the Model for 10 Epochs

BATCH_SIZE = 32
EPOCHS = 10

history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, 
                    validation_data=(X_test, y_test))

(eval_loss, eval_acc) = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=1)

print('\n[INFO] Accuracy: {:.2f}%'.format(eval_acc * 100)) 
print('[INFO] Loss: {}'.format(eval_loss))

end= datetime.datetime.now()
elapsed= end-start

print ('Time: ', elapsed)

In [ ]:
h = history.history
h.keys()

plt.figure(figsize=(8,5))

#plt.subplot(2, 2, 1)
plt.plot(h['accuracy'], label="Training Accuracy")
plt.plot(h['val_accuracy'], label="Validaion Accuracy")
plt.legend(loc='upper left')
#plt.title("Training vs Validation Accuracy")

#plt.subplot(2, 2, 2)
plt.plot(h['loss'], label="Training Loss")
plt.plot(h['val_loss'], label="Validation Loss")
plt.legend(loc='upper right')
#plt.title("Training vs Validation Loss")

plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")

plt.show()

In [ ]:
(_, test_acc) = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=1)
(_, train_acc) = model.evaluate(X_train, y_train, batch_size=BATCH_SIZE, verbose=1)

print('\nTraining Accuracy: {:.2f}%'.format(train_acc * 100)) 
print('Testing Accuracy: {:.2f}%'.format(test_acc * 100))

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
predict_classes = np.argmax(y_pred, axis=1)

In [ ]:
predict_classes

In [ ]:
target_names = ['melanoma', 'nevus', 'seborrheic keratosis']

In [ ]:
print(classification_report(y_test, predict_classes, target_names=target_names))

In [ ]:
cm = confusion_matrix(y_test, predict_classes)

print(cm)

In [ ]:
plt.figure(figsize=(7,5))

plt.title('Confusion Matrix')

sns.heatmap(cm, annot=True, cmap='flare', fmt='d', xticklabels=target_names, yticklabels=target_names)

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')

plt.show()

In [ ]:
idx2 = random.randint(0, len(y_test))

test_img = X_test[idx2, :]
test_label = y_test[idx2]

y_pred = model.predict(X_test[idx2,:].reshape(1, IMAGE_SIZE, IMAGE_SIZE, 3))

y_pred = np.argmax(y_pred)

if (y_pred == 0):
    pred = 'Melanoma'
elif (y_pred == 1):
    pred = 'Nevus'
else:
    pred = 'Seborrheic Keratosis'

    
if (test_label == 0):
    plt.title("Actual Image: Melanoma" +"\nModel Prediction: " + str(pred))
elif (test_label == 1):
    plt.title("Actual Image: Nevus" +"\nModel Prediction: " + str(pred))
else:
    plt.title("Actual Image: Seborrheic Keratosis" +"\nModel Prediction: " + str(pred))
    
plt.imshow(test_img)

plt.show

In [ ]:
model.save('model_skin.h5')

In [ ]:
'''
# wrap our model into a scikit-learn compatible classifier
print("[INFO] initializing model...")

model_fine = KerasClassifier(build_fn=model, verbose=0)

params={'batch_size':[100, 20, 50, 25, 32],  
        'nb_epoch':[200, 100, 300, 400], 
           
        } 
gs=GridSearchCV(estimator=model_fine, param_grid=params, cv=10) 

# now fit the dataset to the GridSearchCV object.  
gs = gs.fit(X_train, y_train)

'''

# Identity Block Inside ResNet50

In [ ]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure   

    Parameters
    ----------
    X : tensor
        input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f : integer
        specifying the shape of the middle CONV's window for the main path
    filters : list
        python list of integers, defining the number of filters in the CONV layers of the main path
    stage : integer
        used to name the layers, depending on their position in the network
    block : str
        used to name the layers, depending on their position in the network

    Returns
    -------
    X : tensor
        output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. we'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', 
               name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', 
               name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', 
               name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

# Convolutional Block in ResNet50

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the convolutional block as defined in Figure   

    Parameters
    ----------
    X : tensor
        input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f : integer
        specifying the shape of the middle CONV's window for the main path
    filters : list
        python list of integers, defining the number of filters in the CONV layers of the main path
    stage : integer
        used to name the layers, depending on their position in the network
    block : str
        used to name the layers, depending on their position in the network
    s : integer, optional
        Integer, specifying the stride to be used. The default is 2.

    Returns
    -------
    X : tensor
        output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    # First component of main path 
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

# Implementing ResNet50

In [ ]:
def ResNet50(input_shape, outputClasses):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Parameters
    ----------
    input_shape : tuple, optional
        shape of the input image. 
    outputClasses : integer, optional
        number of classes. 

    Returns
    -------
    model : object
        a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3 
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL 
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(outputClasses, activation='softmax', name='fc' + str(outputClasses), 
              kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [ ]:
model_rn = ResNet50(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), outputClasses=3)

model_rn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_rn.summary()

In [ ]:
hist = model_rn.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, 
                 validation_data=(X_test, y_test))

In [ ]:
'''
resnet_initial = ResNet50V2(include_top= False)

resnet_initial.trainable = False
'''

In [ ]:
'''
inputs = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

x = resnet_initial(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation = "relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation = "relu")(x)

outputs = layers.Dense(3, activation="softmax")(x)

resnet_model = Model(inputs, outputs)

'''

In [ ]:
'''
resnet_initial.trainable = True

for layer in resnet_initial.layers[:-5]:
    resnet_model.trainable = False

resnet_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
'''

In [ ]:
(_, test_acc) = model_rn.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=1)
(_, train_acc) = model_rn.evaluate(X_train, y_train, batch_size=BATCH_SIZE, verbose=1)

print('\nTraining Accuracy: {:.2f}%'.format(train_acc * 100)) 
print('Testing Accuracy: {:.2f}%'.format(test_acc * 100))

In [ ]:
y_pred_resnet = model_rn.predict(X_test)

In [ ]:
predict_classes_resnet = np.argmax(y_pred_resnet, axis=1)

In [ ]:
print(classification_report(y_test, predict_classes_resnet, target_names=target_names))

In [ ]:
cm = confusion_matrix(y_test, predict_classes_resnet)

plt.figure(figsize=(7,5))

plt.title('Confusion Matrix')

sns.heatmap(cm, annot=True, cmap='flare', fmt='d', xticklabels=target_names, yticklabels=target_names)

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')

plt.show()

In [ ]:
h = hist.history

In [ ]:
plt.figure(figsize=(8,5))

#plt.subplot(2, 2, 1)
plt.plot(h['accuracy'], label="Training Accuracy")
plt.plot(h['val_accuracy'], label="Validaion Accuracy")
plt.legend(loc='upper left')
#plt.title("Training vs Validation Accuracy")

#plt.subplot(2, 2, 2)
plt.plot(h['loss'], label="Training Loss")
plt.plot(h['val_loss'], label="Validation Loss")
plt.legend(loc='upper right')
#plt.title("Training vs Validation Loss")

plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")

plt.show()

# VGG16 Model

In [ ]:
vgg_initial = VGG16(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), weights='imagenet', include_top=False)

In [ ]:
vgg_initial.trainable = False
inputs = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

In [ ]:
x = vgg_initial(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation = "relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation = "relu")(x)

outputs = layers.Dense(3, activation="softmax")(x)

vgg_model = Model(inputs, outputs)
vgg_model.summary()

In [ ]:
vgg_initial.trainable = True

for layer in vgg_initial.layers[:-5]:
    vgg_model.trainable = False

# Make sure you have frozen the correct layers
for i, layer in enumerate(vgg_model.layers):
    print(i, layer.name, layer.trainable)
    
vgg_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
hist = vgg_model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, 
                 validation_data=(X_test, y_test))

In [ ]:
(_, test_acc) = vgg_model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=1)
(_, train_acc) = vgg_model.evaluate(X_train, y_train, batch_size=BATCH_SIZE, verbose=1)

print('\nTraining Accuracy: {:.2f}%'.format(train_acc * 100)) 
print('Testing Accuracy: {:.2f}%'.format(test_acc * 100))

In [ ]:
y_pred_vgg = vgg_model.predict(X_test)

In [ ]:
predict_classes_vgg = np.argmax(y_pred_vgg, axis=1)

In [ ]:
print(classification_report(y_test, predict_classes_vgg, target_names=target_names))

In [ ]:
cm = confusion_matrix(y_test, predict_classes_vgg)

plt.figure(figsize=(7,5))

plt.title('Confusion Matrix')

sns.heatmap(cm, annot=True, cmap='flare', fmt='d', xticklabels=target_names, yticklabels=target_names)

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')

plt.show()

In [ ]:
h = hist.history

In [ ]:
plt.figure(figsize=(8,5))

#plt.subplot(2, 2, 1)
plt.plot(h['accuracy'], label="Training Accuracy")
plt.plot(h['val_accuracy'], label="Validaion Accuracy")
plt.legend(loc='upper left')
#plt.title("Training vs Validation Accuracy")

#plt.subplot(2, 2, 2)
plt.plot(h['loss'], label="Training Loss")
plt.plot(h['val_loss'], label="Validation Loss")
plt.legend(loc='upper right')
#plt.title("Training vs Validation Loss")

plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")

plt.show()